In [21]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from transformers import (AutoModelForCausalLM, AutoTokenizer, BertForSequenceClassification, BertTokenizer, BertModel,
 RobertaForSequenceClassification, RobertaTokenizer, RobertaModel, TrainingArguments, Trainer, RobertaConfig, pipeline, AutoConfig,
 ElectraForSequenceClassification, ElectraTokenizer, utils, AutoModelForSequenceClassification)

from transformers_interpret import SequenceClassificationExplainer
import adapters

# Load models

In [22]:
### DistilBERT mistral ###
detector_path = "distilbert/distilroberta-base"
config = AutoConfig.from_pretrained(detector_path)

# set output_attentions to True to get the attention weights
model_distil = AutoModelForSequenceClassification.from_pretrained(detector_path)
tokenizer_distil = RobertaTokenizer.from_pretrained(detector_path)

model_path = "saved_training_logs_experiment_2/distil_roberta-base/full_finetuning/fake_true_dataset_mistral_10k/15_04_1314/saved_models/best_model.pt"
state_dict = torch.load(model_path)
model_distil.load_state_dict(state_dict)

pipeline_classif_mistral = pipeline('text-classification', model=model_distil, tokenizer=tokenizer_distil)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
### DistilBERT gpt-2 ###

In [ ]:
### DistilBERT gpt-2 ###

In [2]:
### ELECTRA ###
detector_path = "google/electra-large-discriminator"
config = AutoConfig.from_pretrained(detector_path)

# set output_attentions to True to get the attention weights
config.output_attentions = True
model_electra = ElectraForSequenceClassification(config)
tokenizer_electra = ElectraTokenizer.from_pretrained(detector_path)

model_path = "saved_training_logs_experiment_2/electra_large/full_finetuning/fake_true_dataset_mistral_10k/10_04_2148/saved_models/best_model.pt"
state_dict = torch.load(model_path)
model_electra.load_state_dict(state_dict)

<All keys matched successfully>

# Bert Viz base

In [2]:
utils.logging.set_verbosity_error()
detector_path = "google/electra-large-discriminator"
config = AutoConfig.from_pretrained(detector_path)

# set output_attentions to True to get the attention weights
config.output_attentions = True
model = ElectraForSequenceClassification(config)
bert_tokenizer = ElectraTokenizer.from_pretrained(detector_path)

model_path = "saved_training_logs_experiment_2/electra_large/full_finetuning/fake_true_dataset_mistral_10k/10_04_2148/saved_models/best_model.pt"
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [14]:
inputs = bert_tokenizer.encode("Four groups that advocate for immigrant rights said Thursday they will fight to prevent a new law from being enacted. (Published Wednesday, June 18th 2017) — A group of four activists is asking the Supreme Court in an appeal filed on behalf and against one other court case involving undocumented immigrants who have been denied visas because their families are U-Hauled at home or if there's no way around it: The Immigrant Law Enforcement Coalition has requested this week $10 million dollars more ", return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = bert_tokenizer.convert_ids_to_tokens(inputs[0]) 

In [ ]:
from bertviz import head_view
head_view(attention, tokens)

# Captum

In [2]:
#utils.logging.set_verbosity_error()
detector_path = "distilbert/distilroberta-base"
config = AutoConfig.from_pretrained(detector_path)

# set output_attentions to True to get the attention weights
model = AutoModelForSequenceClassification.from_pretrained(detector_path)
tokenizer = RobertaTokenizer.from_pretrained(detector_path)

model_path = "saved_training_logs_experiment_2/distil_roberta-base/full_finetuning/fake_true_dataset_mistral_10k/11_04_1601/saved_models/best_model.pt"
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

pipeline_classif_mistral = pipeline('text-classification', model=model, tokenizer=tokenizer)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import torch
import torch.nn as nn

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig

from captum.attr import IntegratedGradients
from captum.attr import InterpretableEmbeddingBase, TokenReferenceBase
from captum.attr import visualization
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer


# We need to split forward pass into two part: 
# 1) embeddings computation
# 2) classification
def compute_bert_outputs(model_bert, embedding_output, attention_mask=None, head_mask=None):
    if attention_mask is None:
        attention_mask = torch.ones(embedding_output.shape[0], embedding_output.shape[1]).to(embedding_output)

    extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

    extended_attention_mask = extended_attention_mask.to(dtype=next(model_bert.parameters()).dtype) # fp16 compatibility
    extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

    if head_mask is not None:
        if head_mask.dim() == 1:
            head_mask = head_mask.unsqueeze(0).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
            head_mask = head_mask.expand(model_bert.config.num_hidden_layers, -1, -1, -1, -1)
        elif head_mask.dim() == 2:
            head_mask = head_mask.unsqueeze(1).unsqueeze(-1).unsqueeze(-1)  # We can specify head_mask for each layer
        head_mask = head_mask.to(dtype=next(model_bert.parameters()).dtype) # switch to fload if need + fp16 compatibility
    else:
        head_mask = [None] * model_bert.config.num_hidden_layers

    encoder_outputs = model_bert.encoder(embedding_output,
                                         extended_attention_mask,
                                         head_mask=head_mask)
    sequence_output = encoder_outputs[0]
    pooled_output = model_bert.pooler(sequence_output)
    outputs = (sequence_output, pooled_output,) + encoder_outputs[1:]  # add hidden_states and attentions if they are here
    return outputs  # sequence_output, pooled_output, (hidden_states), (attentions)    


class BertModelWrapper(nn.Module):
    
    def __init__(self, model):
        super(BertModelWrapper, self).__init__()
        self.model = model
        
    def forward(self, embeddings):        
        outputs = compute_bert_outputs(self.model.bert, embeddings)
        pooled_output = outputs[1]
        pooled_output = self.model.dropout(pooled_output)
        logits = self.model.classifier(pooled_output)
        return torch.softmax(logits, dim=1)[:, 1].unsqueeze(1)
    
def compute_electra_outputs(model_electra, embedding_output, attention_mask=None, head_mask=None):
    if attention_mask is None:
        attention_mask = torch.ones(embedding_output.shape[0], embedding_output.shape[1]).to(embedding_output)

    extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

    extended_attention_mask = extended_attention_mask.to(dtype=next(model_electra.parameters()).dtype) # fp16 compatibility
    extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

    if head_mask is not None:
        if head_mask.dim() == 1:
            head_mask = head_mask.unsqueeze(0).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
            head_mask = head_mask.expand(model_electra.config.num_hidden_layers, -1, -1, -1, -1)
        elif head_mask.dim() == 2:
            head_mask = head_mask.unsqueeze(1).unsqueeze(-1).unsqueeze(-1)  # We can specify head_mask for each layer
        head_mask = head_mask.to(dtype=next(model_electra.parameters()).dtype) # switch to fload if need + fp16 compatibility
    else:
        head_mask = [None] * model_electra.config.num_hidden_layers

    encoder_outputs = model_electra.encoder(embedding_output,
                                         extended_attention_mask,
                                         head_mask=head_mask)
    sequence_output = encoder_outputs[0]
    outputs = (sequence_output,) + encoder_outputs[1:]  # add hidden_states and attentions if they are here
    return outputs  # sequence_output, (hidden_states), (attentions)    


class ElectraModelWrapper(nn.Module):
    
    def __init__(self, model):
        super(ElectraModelWrapper, self).__init__()
        self.model = model
        
    def forward(self, embeddings):        
        outputs = compute_electra_outputs(self.model.roberta, embeddings)
        sequence_output = outputs[0]
        #sequence_output = self.model.dropout(sequence_output)
        logits = self.model.classifier(sequence_output)
        return torch.softmax(logits, dim=1)[:, 1].unsqueeze(1)
    
electra_model_wrapper = ElectraModelWrapper(model)
ig = IntegratedGradients(electra_model_wrapper)

# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, label=1):

    model.eval()
    model.zero_grad()
    
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)])
    input_embedding = model.roberta.embeddings(input_ids)
    #input_embedding = input_embedding.squeeze(0)

    print("input_embedding", input_embedding.shape)
    
    # predict
    #pred = model_wrapper(input_embedding).item()
    #pred_ind = round(pred)
    pred = pipeline_classif_mistral(sentence)[0]['score']
    pred_ind = round(pred)
    
    

    # compute attributions and approximation delta using integrated gradients
    output = ig.attribute(input_embedding, n_steps=500, return_convergence_delta=True)
    print("Ok after output")
    attributions_ig, delta = ig.attribute(input_embedding, n_steps=500, return_convergence_delta=True)

    print('pred: ', pred_ind, '(', '%.2f' % pred, ')', ', delta: ', abs(delta))

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].numpy().tolist())    
    add_attributions_to_visualizer(attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig)
    
    
def add_attributions_to_visualizer(attributions, tokens, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.detach().numpy()
    
    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            pred_ind,
                            label,
                            "label",
                            attributions.sum(),       
                            tokens[:len(attributions)],
                            delta))    

#interpret_sentence(model, sentence="Four groups that advocate for immigrant rights said Thursday they will fight to prevent a new law from being enacted. (Published Wednesday, June 18th 2017) — A group of four activists is asking the Supreme Court in an appeal filed on behalf and against one other court case involving undocumented immigrants who have been denied visas because their families are U-Hauled at home or if there's no way around it: The Immigrant Law Enforcement Coalition has requested this week $10 million dollars more ", label=1)
#interpret_sentence(model, sentence="Four groups that advocate for immigrant rights said Thursday they will fight to prevent a new law from being enacted. (Published Wednesday, June 18th 2017) — A ", label=1)
#interpret_sentence(model, sentence="Four groups that advocate for immigrant rights said Thursday they will fight to prevent a new law from being enacted. (Published Wednesday, June 18th 2017) — A group of four activists ", label=1)
#interpret_sentence(model, sentence="Four groups that advocate for immigrant rights said Thursday they will fight to prevent a new law from being ", label=1)
interpret_sentence(model, sentence="Four groups that advocate.", label=1)
visualization.visualize_text(vis_data_records_ig)

input_embedding torch.Size([1, 7, 768])
Ok after output
pred:  1 ( 0.99 ) , delta:  tensor([3.4682e-05], dtype=torch.float64)


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (0.99),label,-0.30,#s Four Ġgroups Ġthat Ġadvocate . #/s


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (0.99),label,-0.30,#s Four Ġgroups Ġthat Ġadvocate . #/s


# CLS explainer

In [23]:
cls_explainer = SequenceClassificationExplainer(
    model_distil,
    tokenizer_distil)

mistral_fake_text = "A recent controversy at Baylor University has brought new attention to the issue of whether or not it is appropriate to fire an employee for making a private, non-work related post on Facebook. As we have discussed in the past, employers are allowed to fire employees for their social media activity if that activity violates a written company policy. Baylor University, a private school, has fired an assistant football coach for posting a photo on his personal Facebook page that was deemed offensi"
word_attributions = cls_explainer(mistral_fake_text)

In [24]:
word_attributions

[('<s>', 0.0),
 ('A', 0.008765680010172762),
 ('recent', 0.03564623243912271),
 ('controversy', 0.06019859699808529),
 ('at', 0.021925692169565143),
 ('Baylor', 0.019199058103681436),
 ('University', -0.02770762865638167),
 ('has', 0.09757873623692502),
 ('brought', 0.024249657314594716),
 ('new', -0.02386193581094165),
 ('attention', -0.0025785897373846833),
 ('to', 0.058117931538203035),
 ('the', 0.09720194878147266),
 ('issue', 0.056093902481672196),
 ('of', 0.047652758984337414),
 ('whether', -0.10704245484870814),
 ('or', -0.013887318396272857),
 ('not', 0.016518018743982323),
 ('it', 0.04171437907297394),
 ('is', 0.17617789527987446),
 ('appropriate', -0.09016712173318686),
 ('to', 0.004550017728926134),
 ('fire', 0.08479352762659928),
 ('an', 0.028284509577165367),
 ('employee', 0.18731191203065176),
 ('for', 0.03261639630878221),
 ('making', -0.011517503752739302),
 ('a', 0.07560944905045593),
 ('private', 0.07431907668109616),
 (',', 0.14238366812224384),
 ('non', -0.055448822

In [25]:
word_attributions.sort(key=lambda x: x[1], reverse=True)
word_attributions[:10]

[('was', 0.2992367726405289),
 ('deemed', 0.2626339452763617),
 (',', 0.25855060703632565),
 ('have', 0.23695113249351984),
 ('we', 0.22702153817016496),
 ('are', 0.20806249231478982),
 ('discussed', 0.2065833734341562),
 ('fired', 0.19528286070178677),
 ('employee', 0.18731191203065176),
 ('is', 0.17617789527987446)]

In [26]:
cls_explainer.visualize("bert_viz/distilbert_viz.html")

In [27]:
mistral_true_text = "A recent controversy at Baylor University has brought new attention to the widespread misuse of standardized college admission tests to rank the quality of America's colleges and universities. The SAT and ACT have achieved nearly iconic status in America and throughout the world. As the late author and scholar Stephen Jay Gould noted, the tests are viewed by many as \"abstruse and mysterious,\" with powerful effects on public policy, social mobility, and even individual identity. Misuse of test sc"

word_attributions = cls_explainer(mistral_true_text)

In [28]:
word_attributions.sort(key=lambda x: x[1], reverse=True)
word_attributions[:10]

[('sc', 0.4789958326859266),
 ('stru', 0.18776827171643176),
 ('noted', 0.17458159095791825),
 ('late', 0.12007989203658842),
 ('admission', 0.09755954140482274),
 ('mysterious', 0.08969133066748503),
 ("'s", 0.05661731382316962),
 ('nearly', 0.05621712469305623),
 ('attention', 0.042890995937156555),
 ('widespread', 0.03930890691672416)]

In [30]:
cls_explainer.visualize("bert_viz/distilbert_viz.html")

In [31]:
mistral_fake_text = "Zimbabwean lawmakers on Monday narrowly voted for Lovemore Moyo as Speaker of Parliament, giving the ruling party a majority in the 10-member house. Moyo, 73, who was nominated by President Robert Mugabe's ZANU-PF party, was elected after he got the support of 70 lawmakers out of 108, the state-run Herald newspaper reported. He replaces John Nkomo of the opposition Movement for Democratic Change (MDC) who resigned last week. Nkomo's resignation followed the decision by the MDC to withdraw from t"

word_attributions = cls_explainer(mistral_fake_text)
word_attributions.sort(key=lambda x: x[1], reverse=True)
word_attributions[:10]

[('from', 0.2721676290674081),
 ('was', 0.24507251140361266),
 (',', 0.23595229375715437),
 ('was', 0.22457127860793333),
 ('Moy', 0.22329408039623924),
 ('resigned', 0.21515437962244036),
 ('elected', 0.17296628809494655),
 ('withdraw', 0.15881989493865642),
 ('Parliament', 0.15151324496594243),
 ('.', 0.13782989413535904)]

In [32]:
cls_explainer.visualize("bert_viz/distilbert_viz.html")

In [14]:
mistral_fake_text = "Voters in Japan will turn out for parliamentary elections Sunday to decide the country's future. , a report released by Tokyo-based NGO Save The Children found that more than one million children have been born into poverty since 1999 and nearly half of them are still living with their parents' income at about $25 per day (about US$1) or less – even though they make up only 5 percent - according it has said?s results on Tuesday morning. In its first survey conducted after World War II, which beg"

word_attributions = cls_explainer(mistral_fake_text)
word_attributions.sort(key=lambda x: x[1], reverse=True)
word_attributions[:10]


[('(', 0.38530023937134444),
 ('turn', 0.2993277477220799),
 ('Japan', 0.22801859360212975),
 ('-', 0.21157165614228968),
 ('are', 0.20071580137271772),
 (')', 0.199687891869759),
 ('per', 0.18921770945595995),
 ('elections', 0.17463242336739737),
 ('future', 0.15085602900768155),
 ('for', 0.14572993206353385)]

In [33]:
phi_text_fake = "Four groups that advocate for immigrant rights said Thursday they are working on a new bill that would give undocumented immigrants a path to citizenship, but they said they are still waiting for a final version of the immigration bill passed by the House in November. The bill, known as the American Dream and Promise Act, would allow undocumented immigrants who have lived in the United States for five years and meet other requirements to apply for temporary legal status, followed by a path to ci"

word_attributions = cls_explainer(phi_text_fake)
word_attributions.sort(key=lambda x: x[1], reverse=True)
word_attributions[:10]

[('Thursday', 0.24670920016529832),
 ('but', 0.14702557193162813),
 ('final', 0.1341227944117899),
 ('version', 0.10956783007553174),
 ('Promise', 0.09897758518228991),
 ('House', 0.09519725261213884),
 ('i', 0.0871660357279864),
 ('give', 0.07671664136477782),
 ('Act', 0.07606813619366477),
 ('waiting', 0.06960693536728012)]

In [34]:
cls_explainer.visualize("bert_viz/distilbert_viz.html")

In [35]:
phi_text_fake = "The H1N1 flu virus could cause up to 90,000 U.S. deaths and hundreds of thousands of hospitalizations by spring 2010, according to a report released today by the Centers for Disease Control and Prevention. The new report, the first comprehensive estimate of the potential impact of H1N1 influenza, comes from a model developed by the CDC’s National Center for Immunization and Respiratory Diseases. The model predicts the number of people who will get sick, the number of hospitalizations and deaths "

word_attributions = cls_explainer(phi_text_fake)
word_attributions.sort(key=lambda x: x[1], reverse=True)
word_attributions[:10]

[('Ļ', 0.7688533719592329),
 ('s', 0.34273420076482),
 ('âĢ', 0.309190859236821),
 ('.', 0.1546024692259508),
 (',', 0.12735457156498795),
 (',', 0.10864046940001332),
 ('', 0.09982281749327786),
 ('The', 0.0920830873563357),
 ('The', 0.0836203837961118),
 ('.', 0.07576665631130736)]

In [36]:
cls_explainer.visualize("bert_viz/distilbert_viz.html")